## Imports

In [3]:
import os
import glob

from tqdm import tqdm
import pickle
import random
import numpy as np
import pandas as pd
import nibabel as nib
import scipy.ndimage as sci
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

import tensorflow as tf
from tensorflow import keras
devices = tf.config.list_physical_devices('GPU')
print(devices)
from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical
# from keras_preprocessing.image import ImageDataGenerator

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


## Functions

In [4]:
def crop_3D(img, new_size):
    img_shape = img.shape
    x_mid = int(img_shape[0]/2)
    y_mid = int(img_shape[1]/2)
    z_mid = int(img_shape[2]/2)

    x_diff = int(abs(new_size[0]-x_mid))
    y_diff = int(abs(new_size[1]-y_mid))
    z_diff = int(abs(new_size[2]-z_mid))

    x_start = x_mid-x_diff
    y_start = y_mid-y_diff
    z_start = z_mid-z_diff

    tmp_img = img[x_start:x_start+new_size[0],y_start:y_start+new_size[1],z_start:z_start+new_size[2]]
    return tmp_img

scalar = MinMaxScaler()
def generate_brats_batch(file_pattern, 
                         contrasts, 
                         batch_size=32, 
                         tumour='*', 
                         patient_ids='*',
                         crop_size = (None,None,None), 
                         augment_size=None,
                         infinite = True):
    """
    Generate arrays for each batch, for x (data) and y (labels), where the contrast is treated like a colour channel.
    
    Example:
    x_batch shape: (32, 240, 240, 155, 4)
    y_batch shape: (32, 240, 240, 155)
    
    augment_size must be less than or equal to the batch_size, if None will not augment.
    
    """
    while True:
        n_classes = 4

        # get list of filenames for every contrast available
        keys = dict(prefix=prefix, tumour=tumour)
        filenames_by_contrast = {}
        for contrast in contrasts:
            filenames_by_contrast[contrast] = glob.glob(file_pattern.format(contrast=contrast, patient_id=patient_ids, **keys)) if patient_ids == '*' else []
            if patient_ids != '*':
                contrast_files = []
                for patient_id in patient_ids:
                    contrast_files.extend(glob.glob(file_pattern.format(contrast=contrast, patient_id=patient_id, **keys)))
                filenames_by_contrast[contrast] = contrast_files

        # get the shape of one 3D volume and initialize the batch lists
        arbitrary_contrast = contrasts[0]
        if crop_size == (None,None,None):
            shape = nib.load(filenames_by_contrast[arbitrary_contrast][0]).get_fdata().shape
        else:
            shape = crop_size

        # initialize empty array of batches
        x_batch = np.empty((batch_size, ) + shape + (len(contrasts), )) #, dtype=np.int32)
        y_batch = np.empty((batch_size, ) + shape + (n_classes,)) #, dtype=np.int32)
        num_images = len(filenames_by_contrast[arbitrary_contrast])
        np.random.shuffle(filenames_by_contrast[arbitrary_contrast])
        for bindex in tqdm(range(0, num_images, batch_size), total=num_images):
            filenames = filenames_by_contrast[arbitrary_contrast][bindex:bindex + batch_size]
            for findex, filename in enumerate(filenames):
                for cindex, contrast in enumerate(contrasts):

                    # load raw image batches and normalize the pixels
                    tmp_img = nib.load(filename.replace(arbitrary_contrast, contrast)).get_fdata()
                    tmp_img = scalar.fit_transform(tmp_img.reshape(-1, tmp_img.shape[-1])).reshape(tmp_img.shape)
                    x_batch[findex, ..., cindex] = crop_3D(tmp_img, crop_size)

                    # load mask batches and change to categorical
                    tmp_mask = nib.load(filename.replace(arbitrary_contrast, 'seg')).get_fdata()
                    tmp_mask[tmp_mask==4] = 3
                    tmp_mask = crop_3D(tmp_mask, crop_size)
                    tmp_mask = to_categorical(tmp_mask, num_classes = 4)
                    y_batch[findex] = tmp_mask

            if bindex + batch_size > num_images:
                x_batch, y_batch = x_batch[:num_images - bindex], y_batch[:num_images - bindex]
            if augment_size is not None:
                # x_aug, y_aug = augment(x_batch, y_batch, augment_size)
                x_aug = None
                y_aug = None
                yield np.append(x_batch, x_aug), np.append(y_batch, y_aug)
            else:
                yield x_batch, y_batch
        if not infinite:
            break


## Train Model

In [5]:
if __name__ == '__main__':
    tumours = ['LGG','HGG']
    # prefix = '/Users/jasonfung/Documents/EECE571' # Jason's Macbook
    # prefix = 'C:/Users/Fungj/Documents/EECE_571F' # Jason's Desktop
    brats_dir = '/MICCAI_BraTS_2018_Data_Training/'
    prefix = '/home/atom/Documents/datasets/brats' # Adam's Station
    file_pattern = '{prefix}/MICCAI_BraTS_2018_Data_Training/{tumour}/{patient_id}/{patient_id}_{contrast}.nii.gz'
    # patient_id = 'Brats18_TCIA09_620_1'
    contrasts = ['t1ce', 'flair', 't2']
    tumours = ['LGG', 'HGG']

    data_list_LGG = os.listdir(os.path.join(prefix+brats_dir,tumours[0]))
    data_list_HGG = os.listdir(os.path.join(prefix+brats_dir,tumours[1]))
    dataset_file_list = data_list_HGG + data_list_LGG

    # shuffle and split the dataset file list
    import random
    random.seed(42)
    file_list_shuffled = dataset_file_list.copy()
    random.shuffle(file_list_shuffled)
    test_ratio = 0.2

    train_file, test_file = file_list_shuffled[0:int(len(file_list_shuffled)*(1-test_ratio))], file_list_shuffled[int(len(file_list_shuffled)*(1-test_ratio)):]
    
    if '.DS_Store' in train_file:
        train_file.remove('.DS_Store')

    batch_size = 2
    train_datagen = generate_brats_batch(file_pattern, contrasts, batch_size = batch_size, patient_ids = train_file , crop_size= (128,128,128)) # first iteration
    test_datagen = generate_brats_batch(file_pattern, contrasts, batch_size = batch_size, patient_ids = test_file, crop_size= (128,128,128)) # first iteration

    from custom_unet import *
    import segmentation_models_3D as sm 
    sm.set_framework('tf.keras')

    # data parameters
    x_size = 128
    y_size = 128
    z_size = 128
    contrast_channels = 3
    input_shape = (x_size, y_size, z_size, contrast_channels)
    n_classes = 4

    # define Hyper Parameters
    LR = 0.0001
    activation = 'softmax'

    encoder_weights = 'imagenet'
    BACKBONE = 'vgg16'
    optim = tf.keras.optimizers.Adam(LR)
    class_weights = [0.25, 0.25, 0.25, 0.25]

    # limit memory growth
    # gpus = tf.config.experimental.list_physical_devices('GPU')
    # tf.config.experimental.set_memory_growth(gpus[0], True)

    # Define Loss Functions
    dice_loss = sm.losses.DiceLoss(class_weights=class_weights)
    focal_loss = sm.losses.CategoricalFocalLoss()
    total_loss = dice_loss + (1*focal_loss)
    metrics = [sm.metrics.IOUScore(threshold = 0.5)]

    # Define the model being used. In this case, UNet
    model = unet_model((x_size,y_size,z_size,contrast_channels), 
                        n_classes, 
                        dropout = 0.1, 
                        max_pooling = True)

    model.compile(optimizer = optim, loss = total_loss, metrics = metrics)

    steps_per_epoch = len(train_file)//batch_size
    val_steps_per_epoch = len(test_file)//batch_size
    
    
    my_callbacks = [tf.keras.callbacks.EarlyStopping(patience = 5),
                    tf.keras.callbacks.TensorBoard(log_dir = prefix + 'models/unet' + './logs'),
                    tf.keras.callbacks.ModelCheckpoint(filepath = prefix + '/unet_model_20220321.h5', monitor = 'val_loss', save_best_only = True)
                   ]
                    
    with tf.device('/device:GPU:0'):
        history = model.fit(train_datagen,
                            steps_per_epoch = steps_per_epoch,
                            epochs = 8,
                            verbose = 1,
                            validation_data = test_datagen,
                            validation_steps = val_steps_per_epoch,
                            callbacks = my_callbacks)


2022-04-01 15:27:02.898908: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-04-01 15:27:02.899462: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-01 15:27:02.899858: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-01 15:27:02.900205: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

softmax
(None, 128, 128, 128, 3)
(None, 128, 128, 128, 4)
Segmentation Models: using `tf.keras` framework.
softmax


  0%|                                                                                                     | 0/227 [00:00<?, ?it/s]

Epoch 1/8


  0%|▍                                                                                            | 1/227 [00:04<15:09,  4.02s/it]2022-04-01 15:27:08.617309: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8302
2022-04-01 15:27:12.296274: W tensorflow/core/common_runtime/bfc_allocator.cc:275] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.02GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2022-04-01 15:27:12.296300: W tensorflow/core/kernels/gpu_utils.cc:50] Failed to allocate memory for convolution redzone checking; skipping this check. This is benign and only means that we won't check cudnn for out-of-bounds reads and writes. This message will only be printed once.
2022-04-01 15:27:12.296483: W tensorflow/core/common_runtime/bfc_allocator.cc:275] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.05GiB with freed_by_count=0. The 

...

 92/114 [=======================>......] - ETA: 36s - loss: 0.9946 - iou_score: 0.0797

 41%|█████████████████████████████████████▋                                                      | 93/227 [03:26<03:44,  1.67s/it]

 93/114 [=======================>......] - ETA: 35s - loss: 0.9944 - iou_score: 0.0808

 41%|██████████████████████████████████████                                                      | 94/227 [03:28<03:42,  1.67s/it]2022-04-01 15:34:00.243953: W tensorflow/core/common_runtime/bfc_allocator.cc:462] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.00GiB (rounded to 1073741824)requested by op gradient_tape/u-net/conv3d_35/Conv3D/Conv3DBackpropInputV2
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2022-04-01 15:34:00.244072: I tensorflow/core/common_runtime/bfc_allocator.cc:1010] BFCAllocator dump for GPU_0_bfc
2022-04-01 15:34:00.244113: I tensorflow/core/common_runtime/bfc_allocator.cc:1017] Bin (256): 	Total Chunks: 239, Chunks in use: 236. 59.8KiB allocated for chunks. 59.0KiB in use in bin. 28.6KiB client-requested in use in bin.
2022-04-01 15:34:00.244141: I tensorflow/core/common_runtime/bfc_allocat

ResourceExhaustedError: Graph execution error:

Detected at node 'gradient_tape/u-net/conv3d_35/Conv3D/Conv3DBackpropInputV2' defined at (most recent call last):
    File "/home/atom/.miniconda/lib/python3.9/runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/home/atom/.miniconda/lib/python3.9/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/home/atom/.miniconda/lib/python3.9/site-packages/ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "/home/atom/.miniconda/lib/python3.9/site-packages/traitlets/config/application.py", line 846, in launch_instance
      app.start()
    File "/home/atom/.miniconda/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 677, in start
      self.io_loop.start()
    File "/home/atom/.miniconda/lib/python3.9/site-packages/tornado/platform/asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File "/home/atom/.miniconda/li